In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
#데이터셋 로더
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
#음성파일 처리
import librosa
#기타
import numpy as np
import os

In [7]:
# 패딩 함수
padding = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

In [ ]:
from pickle import TRUE
from tqdm import tqdm
# 전처리
from PIL import Image

# 0 에어컨
# 2 children_playing
# 3 dog_bark
# 4 drill
# 6 gunshot
# 7 jack hammer
# 9 street music
skip_class=['0', '2', '3', '4', '6', '7', '9']

def pre_progressing(sound_dir, image_dir):
  # 마지막에 /로 끝나게
  if image_dir[-1] != '/':
    image_dir.join('/')
  if sound_dir[-1] != '/':
    sound_dir.join('/')
  for filename in tqdm(os.listdir(sound_dir)):
    # print(filename)
    # 유니코드 정규화?
    # filename = normalize('NFC', filename)
    # wav 포맷 데이터만 사용

    if '.wav' not in filename:
      continue
    # 필요없는 클래스 스
    if filename.split('-')[1] in skip_class:
      continue
    wav, sr = librosa.load(sound_dir+filename, sr=16000)
    # 이미지 ndarray
    mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=100, n_fft=400, hop_length=160)
    # 패딩된 ndarray
    # 리사이즈 할거면 패딩 필요없음?
    padded_mfcc = padding(mfcc, 500)

    # 해당 이미지 새로 저장
    # img = Image.fromarray(padded_mfcc)
    img = Image.fromarray(mfcc)
    img = img.convert('L')
    # resize크기 적당히 조절
    img = img.resize([32,32])
    img.save(image_dir+filename.split('-')[1]+'/'+filename+'.jpg','JPEG')

# 돌리기 전에 폴더 비워둘것
# 클래스 빈 폴더 생성
for i in range(10):
  os.makedirs('/content/drive/MyDrive/gist_공모전/archive-2/image_set/'+str(i), exist_ok=True)
# 전처리 과정 수행
for i in range(10):
  pre_progressing('/content/drive/MyDrive/gist_공모전/archive-2/fold'+str(i+1)+'/', '/content/drive/MyDrive/gist_공모전/archive-2/image_set/')


 86%|████████▌ | 807/936 [03:39<01:15,  1.72it/s]

In [ ]:
for class_folder in os.listdir('/content/drive/MyDrive/gist_공모전/archive-2/image_set/'):
  if len(os.listdir('/content/drive/MyDrive/gist_공모전/archive-2/image_set/'+class_folder)) == 0:
    os.rmdir('/content/drive/MyDrive/gist_공모전/archive-2/image_set/'+class_folder)

FileNotFoundError: [Errno 2] No such file or directory: '/content/mount/MyDrive/image_set/'

In [ ]:
dataset = ImageFolder(root='/content/drive/MyDrive/gist_공모전/archive-2/image_set/',transform=transforms.Compose([transforms.ToTensor(),]))

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 각각의 데이터셋에 대해 DataLoader 생성
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
#GPT코드
#mobilenet 버전
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import matplotlib.pyplot as plt

# 1. MobileNetV2 모델 불러오기 (pretrained 모델 사용 가능)
model = models.mobilenet_v2(pretrained=True)  # 미리 학습된 가중치 사용
# 만약 MobileNetV3를 사용하려면: models.mobilenet_v3_large(pretrained=True)

# 2. 모델의 마지막 레이어를 학습할 데이터에 맞게 수정 (예: 10개 클래스라면 10으로 수정)
num_classes = len(os.listdir('/content/mount/MyDrive/image_set/'))  # 예시: 클래스가 10개인 경우
model.classifier[1] = nn.Linear(model.last_channel, num_classes)  # 마지막 레이어 수정

# 3. 모델을 GPU로 보내기 (가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 4. 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 학습 함수 정의
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 순전파 (Forward pass)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 역전파 (Backward pass) 및 최적화
        loss.backward()
        optimizer.step()

        # 손실 계산
        running_loss += loss.item()

        # 정확도 계산
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels).item()
        total_predictions += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct_predictions / total_predictions

    return epoch_loss, epoch_acc

# 6. 검증 함수 정의
def validate_model(model, val_loader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # 검증 시에는 그래디언트를 계산하지 않음
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # 순전파
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 손실 계산
            running_loss += loss.item()

            # 정확도 계산
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels).item()
            total_predictions += labels.size(0)

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = correct_predictions / total_predictions
    return epoch_loss, epoch_acc

# 7. 학습 및 검증 루프 실행 및 평가지표 저장
num_epochs = 100
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss, train_acc = train_model(model, train_data_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_model(model, val_data_loader, criterion, device)

    # 각 epoch의 손실과 정확도를 저장
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
    print(f'Validation Loss: {val_loss:.4f}, Validation Acc: {val_acc:.4f}')

print("Training complete.")
torch.save(model,'/content/drive/MyDrive/gist_공모전/model_parameter' )

# 8. 학습 및 검증 결과 그래프 그리기
epochs = range(1, num_epochs + 1)

# Loss 그래프
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'bo-', label='Train Loss')
plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
plt.title('Train and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy 그래프
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'bo-', label='Train Accuracy')
plt.plot(epochs, val_accuracies, 'ro-', label='Validation Accuracy')
plt.title('Train and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
model = torch.load('/content/drive/MyDrive/gist_공모전')

In [ ]:
#testset_pre_progressing

def padding(mfcc, max_len):
    # mfcc 배열을 최대 길이에 맞게 패딩 (길이가 부족하면 0으로 채움)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    return mfcc

def ensure_directory_exists(directory):
    # 디렉터리가 존재하지 않으면 생성
    if not os.path.exists(directory):
        os.makedirs(directory)

def testset_pre_progressing(sound_dir, image_dir):
    for filename in tqdm(os.listdir(sound_dir)):
        # .wav 파일만 처리
        if not filename.endswith('.wav'):
            continue

        # 오디오 파일 로드
        wav, sr = librosa.load(os.path.join(sound_dir, filename), sr=16000)

        # MFCC 특징 추출
        mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=100, n_fft=400, hop_length=160)

        # 패딩된 MFCC
        padded_mfcc = padding(mfcc, 500)

        # 이미지로 변환
        img = Image.fromarray(padded_mfcc)
        img = img.convert('L')
        img = img.resize((32, 32))

        # 파일 이름에서 클래스를 추출하고, 저장할 디렉터리 경로 생성
        class_dir = filename.split('-')[1]
        save_dir = os.path.join(image_dir, class_dir)

        # 디렉터리가 존재하지 않으면 생성
        ensure_directory_exists(save_dir)

        # 이미지 저장 경로 생성
        save_path = os.path.join(save_dir, filename.replace('.wav', '.jpg'))

        # 이미지 저장
        img.save(save_path, 'JPEG')
testset_pre_progressing('/content/drive/MyDrive/gist_공모전/ESC-50-master/audio', '/content/mount/MyDrive/gist_공모전/ESC-50-master/image_set')

100%|██████████| 1006/1006 [01:14<00:00, 13.43it/s]


In [ ]:
#none_dataset 준비
none_dataset = ImageFolder(root = '/content/mount/MyDrive/gist_공모전/ESC-50-master/image_set', transform = transforms.Compose([transforms.ToTensor(),]))
none_dataset_loader = DataLoader(none_dataset, batch_size=32, shuffle = True)